<a id="toc"></a>
# M1 mirror system redesign

[rafael.celestre@synchrotron-soleil.fr](mailto:rafael.celestre@synchrotron-soleil.fr)

Contents :
1. [Reflection spectra of commonly used coatings](#reflec_curves)
1. [Coating Candidates](#coating-candidates)

***

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '21/05/2024'
__changed__ = '21/10/2024'

import sys

import numpy as np
from barc4plots.barc4plots import PlotManager
from barc4xoc.materials import reflectivity_curve, reflectivity_map, combine_mirrors
from barc4xoc.misc import energy_wavelength

# checking we are using the correct python:
print(sys.executable)
print(sys.version)

# benchmarking tools
# %load_ext autoreload
# %autoreload 2
# %matplotlib widget


<a id="reflec_curves"></a>
## Reflection spectra of commonly used coatings
[Back to the top](#toc)


In [2]:
e0, ei, ef, dE = 0, 250, 5000, 5
ne = int((ef-ei)/dE)+1
energy = np.linspace(ei, ef, ne)

In [3]:
theta0, thetai, thetaf, dtheta = 0.0, 0.25, 3.0, 0.01
ntheta = int((thetaf-thetai)/dtheta)+1
theta = np.linspace(thetai, thetaf, ntheta)
reflec_cutoff = 5/100

In [ ]:
mirror_length = 1/np.sin(np.deg2rad(theta))

fig = PlotManager()

fig.image, fig.x = mirror_length, theta
fig.additional_info("mirror length per beam unit size", "grazing angle [°]", "[mm/mm]", 
                    xmin=theta0, xmax=thetaf+(thetai-theta0), ymin=0, ymax=250).aesthetics(LaTex=True, grid=True).info_1d_plot(0).plot_1d()

In [5]:
def plot_reflec_map(reflec_dict, reflec_cutoff=0, cm=3):
    
    reflec_map = reflec_dict["reflectivity"]
    if reflec_cutoff != 0:
        reflec_map[reflec_map<reflec_cutoff] = np.nan
    fig = PlotManager()
    fig.image, fig.x, fig.y = reflec_map, reflec_dict["energy"], reflec_dict["theta"]
    fig.additional_info(reflec_dict["material"], "energy [eV]", "grazing angle [°]", e0, ef+(ei-e0), theta0, thetaf+(thetai-theta0), 0, 1,
                        ).aesthetics(LaTex=True, grid=True, AspectRatio=False).info_2d_plot(cm).plot_2d()

### Low Z

In [ ]:
plot_reflec_map(reflectivity_map("B4C", 2.52, thetai, thetaf, ntheta, ei, ef, ne), reflec_cutoff)

### Intermediate Z

In [ ]:
plot_reflec_map(reflectivity_map("Mo", -1, thetai, thetaf, ntheta, ei, ef, ne), reflec_cutoff)

In [ ]:
plot_reflec_map(reflectivity_map("Rh", -1, thetai, thetaf, ntheta, ei, ef, ne), reflec_cutoff)

### High Z

In [ ]:
plot_reflec_map(reflectivity_map("Pt", -1, thetai, thetaf, ntheta, ei, ef, ne), reflec_cutoff)

<a id="coating_candidates"></a>
# Coating candidates
[Back to the top](#toc)

Incident anglesshould be (ideally) between $\theta=0.6°$ and 
$\theta=2.5°$.

In [10]:
def plot_mirror_combinations(transmission, combination, energy, theta):
    
    fig = PlotManager()

    plot_configs = [
        {"index": k, "image": transmission[:, k], "x": energy, "hold": (k != 0),
        "title": f'{combination[k, 0]} vs. {combination[k, 1]}',
        "enable": (k == transmission.shape[1] - 1)}
        for k in range(transmission.shape[1])]

    for config in plot_configs:
        fig.image, fig.x = config["image"], config["x"]
        if config["index"] == 0:
            fig.additional_info(f'grazing angle @ {theta:.2f}°', "energy [eV]", "reflectivity", 
                                xmin=e0, xmax=ef+(ei-e0), ymin=-0.05, ymax=1.05).aesthetics(LaTex=True, grid=True)
        fig.info_1d_plot(config["index"], config["title"]).plot_1d(enable=config["enable"], hold=config["hold"])

In [11]:
ei = 250
ef = 10000
ne = int((ef-ei)/0.2)

## M1A and M1B/C (low energy configuration)

In [ ]:
theta_scan = [2.5, 2.0, 1.5, 1.0]

for theta in theta_scan:
    candidates = [reflectivity_curve("B4C", 2.52, theta, ei, ef, ne), 
                reflectivity_curve("Rh", -1, theta, ei, ef, ne),
                reflectivity_curve("Pt", -1, theta, ei, ef, ne)]
    transmission1, combination = combine_mirrors(candidates, 2)
    plot_mirror_combinations(transmission1, combination, np.linspace(ei, ef, ne), theta)

## M1A and M1B/C (high energy configuration)

In [ ]:
theta_scan = [1.0, 0.8, 0.6, 0.4]

for theta in theta_scan:
    candidates = [reflectivity_curve("B4C", 2.52, theta, ei, ef, ne), 
                reflectivity_curve("Rh", -1, theta, ei, ef, ne),
                reflectivity_curve("Pt", -1, theta, ei, ef, ne)]
    transmission1, combination = combine_mirrors(candidates, 2)
    plot_mirror_combinations(transmission1, combination, np.linspace(ei, ef, ne), theta)

## M1A and M1B/C angles and coating choices

In [ ]:
theta=1.5
c1 = reflectivity_curve("B4C", 2.52, theta, ei, ef, ne)
c2 = reflectivity_curve("Pt", -1, theta, ei, ef, ne)

In [ ]:
fig = PlotManager()

fig.image, fig.x = c1["reflectivity"]*c2["reflectivity"], c1["energy"]
fig.additional_info("M1 throughput", "energy [eV]", "efficiency ($\eta$)", xmin=e0, xmax=ef+(ei-e0), ymin=-0.03, ymax=1.03).aesthetics(LaTex=True, grid=True)
fig.info_1d_plot(0, f'{c1["material"]} $\\times$ {c2["material"]} @ {c1["theta"]:.2f}°').plot_1d(enable=False)
fig.image, fig.x = c1["reflectivity"]*c1["reflectivity"], c1["energy"]
fig.info_1d_plot(2, f'{c1["material"]} $\\times$ {c1["material"]} @ {c1["theta"]:.2f}°').plot_1d(enable=True, hold=True)


In [ ]:
theta=0.43
c1 = reflectivity_curve("B4C", 2.52, theta, ei, ef, ne)
c2 = reflectivity_curve("Pt", -1, theta, ei, ef, ne)

In [ ]:
fig = PlotManager()

fig.image, fig.x = c1["reflectivity"]*c2["reflectivity"], c1["energy"]
fig.additional_info("M1 throughput", "energy [eV]", "efficiency ($\eta$)", xmin=e0, xmax=ef+(ei-e0), ymin=-0.03, ymax=1.03).aesthetics(LaTex=True, grid=True)
fig.info_1d_plot(0, f'{c1["material"]} $\\times$ {c2["material"]} @ {c1["theta"]:.2f}°').plot_1d(enable=False)
fig.image, fig.x = c1["reflectivity"]*c1["reflectivity"], c1["energy"]
fig.info_1d_plot(2, f'{c1["material"]} $\\times$ {c1["material"]} @ {c1["theta"]:.2f}°').plot_1d(enable=True, hold=True)

## BOREAS type solution

In [ ]:
theta=0.43
c1 = reflectivity_curve("Au", -1, 1.0, ei, ef, ne)
c2 = reflectivity_curve("Ni", -1, 1.0, ei, ef, ne)
c3 = reflectivity_curve("Au", -1, 1.15, ei, ef, ne)

In [ ]:
fig.image, fig.x = c1["reflectivity"]*c3["reflectivity"], c1["energy"]
fig.additional_info("M1 (BOREAS)", "energy [eV]", "efficiency ($\eta$)", xmin=e0, xmax=ef+(ei-e0), ymin=-0.03, ymax=1.03).aesthetics(LaTex=True, grid=True)
fig.info_1d_plot(0, f'{c1["material"]} $\\times$ {c3["material"]}').plot_1d(enable=False)
fig.image, fig.x = c2["reflectivity"]*c3["reflectivity"], c1["energy"]
fig.info_1d_plot(2, f'{c2["material"]} $\\times$ {c3["material"]}').plot_1d(enable=True, hold=True)